## Try requests and bs4

In [2]:
import requests
from bs4 import BeautifulSoup

In [3]:
app_url = 'https://finance.yahoo.com/quote/APP/holders/'

In [4]:
agent = 'Mozilla/5.0 (Windows NT 10.0; Windows; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.5060.114 Safari/537.36'

In [5]:
response = requests.get(app_url, headers={'User-Agent': agent})
response

<Response [200]>

In [6]:
soup = BeautifulSoup(response.text, 'html.parser')

In [7]:
# Locate the section containing the "Major Holders" table by the section's `data-testid` attribute
section = soup.find('section', {'data-testid': 'holders-major-holders-table'})

In [8]:
# Find all 'td' elements within the section to extract the data
td_elements = section.find_all('td', class_='majorHolders')
td_elements

[<td class="majorHolders yf-1toamfi">27.69% </td>,
 <td class="majorHolders yf-1toamfi">% of Shares Held by All Insider </td>,
 <td class="majorHolders yf-1toamfi">60.85% </td>,
 <td class="majorHolders yf-1toamfi">% of Shares Held by Institutions </td>,
 <td class="majorHolders yf-1toamfi">84.15% </td>,
 <td class="majorHolders yf-1toamfi">% of Float Held by Institutions </td>,
 <td class="majorHolders yf-1toamfi">861 </td>,
 <td class="majorHolders yf-1toamfi">Number of Institutions Holding Shares </td>]

In [9]:
# Initialize an empty dictionary
holders_dict = {}

# Iterate through the list in pairs (numeric value and its corresponding label)
for i in range(0, len(td_elements), 2):
    # Strip any extra characters and clean up the value
    value = td_elements[i].text.strip().replace('%', '')
    
    # Convert the value to float or integer as appropriate
    if '.' in value:
        value = float(value)
    else:
        value = int(value)
    
    # Get the corresponding key (label)
    label = td_elements[i+1].text.strip()
    
    # Update the dictionary
    holders_dict[label] = value

In [10]:
holders_dict

{'% of Shares Held by All Insider': 27.69,
 '% of Shares Held by Institutions': 60.85,
 '% of Float Held by Institutions': 84.15,
 'Number of Institutions Holding Shares': 861}

## Send to gsheet

### Test gspread connection

In [11]:
import gspread

sa_path = r"C:\Users\Owner\AppData\Local\Programs\Python\Python310\Lib\site-packages\gspread\service_account.json"
gc = gspread.service_account(filename=sa_path)

sh = gc.open("APP Ownership")

print(sh.sheet1.get('A1'))

[['% of Shares Held by All Insider']]


### Send data to spreadsheet

In [30]:
import gspread
from datetime import datetime

sa_path = r"C:\Users\Owner\AppData\Local\Programs\Python\Python310\Lib\site-packages\gspread\service_account.json"
gc = gspread.service_account(filename=sa_path)

In [31]:
# Open the sheet
sh = gc.open("APP Ownership")
worksheet = sh.worksheet("Data")  # Access the "Data" sheet

#### next available row

In [32]:
# Function to find the next empty row
def next_available_row(worksheet):
    str_list = list(filter(None, worksheet.col_values(1)))  # Get the values in column A
    return len(str_list) + 1  # Find the next empty row by checking how many rows are filled

In [42]:
# Extract the values in the correct order (A, B, C, D)
values = [
    holders_dict["% of Shares Held by All Insider"],
    holders_dict["% of Shares Held by Institutions"],
    holders_dict["% of Float Held by Institutions"],
    holders_dict["Number of Institutions Holding Shares"],
    datetime.now().strftime("%Y/%m/%d %H:%M:%S") + " MT"
]

In [39]:
# Find the next available row
next_row = next_available_row(worksheet)

In [40]:
# Update the next empty row with the values in columns A to D
worksheet.update(f'A{next_row}:E{next_row}', [values])

C:\Users\Owner\AppData\Local\Temp\ipykernel_908\1879940049.py:2: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  worksheet.update(f'A{next_row}:E{next_row}', [values])


{'spreadsheetId': '1WLCqIuF9P7_ypl_hF-9QtwaFr2-jjl0DDNgpB9HfB7A',
 'updatedRange': 'Data!A2:E2',
 'updatedRows': 1,
 'updatedColumns': 5,
 'updatedCells': 5}

#### insert a new row, move others down

In [44]:
# Insert a new row at position 2 (shifts existing row 2 and below down)
worksheet.insert_row([], 2)

{'spreadsheetId': '1WLCqIuF9P7_ypl_hF-9QtwaFr2-jjl0DDNgpB9HfB7A',
 'updates': {'spreadsheetId': '1WLCqIuF9P7_ypl_hF-9QtwaFr2-jjl0DDNgpB9HfB7A',
  'updatedRange': 'Data!A2'}}

In [46]:
# Extract the values in the correct order (A, B, C, D)
values = [
    holders_dict["% of Shares Held by All Insider"],
    holders_dict["% of Shares Held by Institutions"],
    holders_dict["% of Float Held by Institutions"],
    holders_dict["Number of Institutions Holding Shares"],
    datetime.now().strftime("%Y/%m/%d %H:%M:%S") + " MT"
]

In [47]:
# Update row 2 with the new data in columns A to D
worksheet.update('A2:E2', [values])

C:\Users\Owner\AppData\Local\Temp\ipykernel_908\2866740410.py:2: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  worksheet.update('A2:E2', [values])


{'spreadsheetId': '1WLCqIuF9P7_ypl_hF-9QtwaFr2-jjl0DDNgpB9HfB7A',
 'updatedRange': 'Data!A2:E2',
 'updatedRows': 1,
 'updatedColumns': 5,
 'updatedCells': 5}